In [4]:
import argparse
import copy
import logging
import math
import os

import keras
import numpy as np
import tensorflow as tf

from cleverhans.attacks import DeepFool, MomentumIterativeMethod
from cleverhans.utils import batch_indices
from cleverhans.utils_keras import KerasModelWrapper
from utils import (fully_connected_nn_model, load_data, to_onehot)



In [5]:
num_hidden_layers=2
noise_eps=0.1
lr=0.001
epochs=2
batch_size=128
num_noises=3
num_oracle_iter=2
data='mnist'


In [6]:

# Load data
X_train, Y_train, X_test, Y_test = load_data(data)
num_train, num_rows, num_cols, num_channels = X_train.shape
_, num_classes = Y_train.shape

# Load adversarial examples generated from logistic regressions
X_train_adv = np.zeros((num_noises, num_train, num_rows, num_cols,
                        num_channels))
X_train_adv[0] = X_train
for i in range(num_noises):
    X_train_adv[i] = np.load(f"../data/{data}/logreg_adv/adv_{i}.npy")

sess = tf.Session()
sess.run(tf.global_variables_initializer())

x = tf.placeholder(
    tf.float32,
    shape=(num_noises, None, num_rows, num_cols, num_channels))
y = tf.placeholder(tf.float32, shape=(None, num_classes))
w = tf.placeholder(tf.float32, shape=(num_noises, ))

model = fully_connected_nn_model(
    (num_rows, num_cols, num_channels),
    num_hidden_layers=num_hidden_layers)
wrap = KerasModelWrapper(model, num_classes)


In [24]:
losses = tf.zeros([0], tf.float32)


In [25]:
losses = tf.zeros([0], tf.float32)

def body(i, losses):
    logits = wrap.get_logits(x[i])
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))
    losses = tf.concat([losses, [loss]], 0)
    return i + 1, losses


def condition(i, losses):
    return i < num_noises


_, losses = tf.while_loop(condition, body, [0, losses], 
                        shape_invariants=[tf.TensorShape(None),tf.TensorShape([None])])

In [28]:
loss = w * losses

In [30]:
# Define the update func
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_step = optimizer.minimize(loss)

logits = wrap.get_logits(x[0])
acc, acc_op = tf.metrics.accuracy(
    labels=tf.argmax(y, 1), predictions=tf.argmax(logits, 1))



In [31]:
# Define adv attack
deepfool = DeepFool(wrap, sess=sess)
deepfool_params = {'eps': noise_eps, 'clip_min': 0., 'clip_max': 1.}

# Attack images
x_deepfool = deepfool.generate(x[0], **deepfool_params)
# Consider the attack to be constant
x_deepfool = tf.stop_gradient(x_deepfool)

# Evaluate predictions on adv attacks
preds_deepfool = model(x_deepfool)
acc_deepfool, acc_op_deepfool = tf.metrics.accuracy(
    labels=tf.argmax(y, 1), predictions=tf.argmax(preds_deepfool, 1))

# Define adv attack
momentum_iterative = MomentumIterativeMethod(wrap, sess=sess)
momentum_iterative_params = {
    'eps': noise_eps,
    'clip_min': 0.,
    'clip_max': 1.
}

# Attack images
x_momentum_iterative = momentum_iterative.generate(x[0], **deepfool_params)
# Consider the attack to be constant
x_momentum_iterative = tf.stop_gradient(x_momentum_iterative)

# Evaluate predictions on adv attacks
preds_momentum_iterative = model(x_momentum_iterative)
acc_momentum_iterative, acc_op_momentum_iterative = tf.metrics.accuracy(
    labels=tf.argmax(y, 1), predictions=tf.argmax(preds_momentum_iterative, 1))

saver = tf.train.Saver()
rng = np.random.RandomState()  # for batch sampling


Instructions for updating:
keep_dims is deprecated, use keepdims instead


07/22/2018 11:17:36 PM: From /Users/kojin/projects/robust-opt-adversarial/cleverhans/cleverhans/attacks.py:563: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


07/22/2018 11:17:37 PM: From /Users/kojin/projects/robust-opt-adversarial/cleverhans/cleverhans/attacks.py:585: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [35]:
# Each row is w_t, simplex vector over noises
weights_distribution = np.full((num_oracle_iter, num_noises),
                               1. / num_noises)
# Has value L_i(x_t) for each i in noises and t in oracle_iter
losses_np = np.zeros((num_oracle_iter, num_noises))

# Number of times the Bayesian oracle is invoked
for oracle_iter in range(num_oracle_iter):
    logging.info(f'oracle_iter: {oracle_iter}')
    # initialize variables every iteration
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    ### Compute the weights for the distributional oracle for this iteration ###
    # eta is the time dependent parameter controlling the weight distribution
    eta = np.sqrt(np.log(num_noises) / (2 * num_oracle_iter))
    # See Algorithm 1 (3) for this udpate
    unnormalized_current_weights = np.exp(
        eta * losses_np[0:oracle_iter, :].sum(axis=0))
    normalized_current_weights = unnormalized_current_weights / np.sum(
        unnormalized_current_weights)
    # Log current weights for later iterations
    weights_distribution[oracle_iter, :] = normalized_current_weights
    logging.info(f"Noise ratio: f{normalized_current_weights}")

    ### Train model with weighted loss for each noise ###
    loss_total_list = np.zeros((num_noises, ))
    for epoch in range(epochs):
        logging.info(f'epoch: {epoch}')
        # Compute number of batches
        num_batches = int(math.ceil(num_train / batch_size))
        assert num_batches * batch_size >= num_train

        # Indices to shuffle training set
        index_shuf = list(range(num_train))
        rng.shuffle(index_shuf)
        for batch in range(num_batches):
            # Compute batch start and end indices
            start, end = batch_indices(batch, num_train, batch_size)

            feed_dict = {
                x: X_train_adv[:, index_shuf[start:end]],
                y: Y_train[index_shuf[start:end]],
                w: weights_distribution[oracle_iter]
            }
            loss_vals, _ = sess.run([losses, train_step], feed_dict=feed_dict)
            # Normalize and log loss
            loss_total_list += loss_vals / (epochs * num_batches)
    # Save average loss for the next iteration
    losses_np[oracle_iter, :] = loss_total_list

# Save the last trained model
saver.save(sess, "../model/robust_optimization/model.ckpt")

# 1. Accuracy on uncorrupted test set
feed_dict = {x: X_test, y: Y_test}
test_acc = sess.run(acc_op, feed_dict=feed_dict)
logger.info(f"Test Acc: {test_acc}")

# 2. Accuracy on test set corrupted by known noises that are used during training
test_acc_deepfool = sess.run(acc_op_deepfool, feed_dict=feed_dict)
logger.info(f"Test Acc Deepfool: {test_acc_deepfool}")

# 3. Accuracy on test set corrupted by adversarial noises not used during training
test_acc_momentum_iterative = sess.run(
    acc_op_momentum_iterative, feed_dict=feed_dict)
logger.info(f"Test Acc Momentum Iterative: {test_acc_momentum_iterative}")

# Log experiment result
path = f'../result/robust_optimization'
os.makedirs(path, exist_ok=True)
losses.dump(f'{path}/losses.npy')
weights_distribution.dump(f'{path}/weights_distribution.npy')
accs = np.array([test_acc, test_acc_deepfool, test_acc_momentum_iterative])
accs.dump(f'{path}/accs.npy')

07/22/2018 11:19:30 PM: oracle_iter: 0
07/22/2018 11:19:30 PM: Noise ratio: f[ 0.33333333  0.33333333  0.33333333]
07/22/2018 11:19:30 PM: epoch: 0
07/22/2018 11:20:17 PM: epoch: 1
07/22/2018 11:20:50 PM: oracle_iter: 1
07/22/2018 11:20:50 PM: Noise ratio: f[ 0.33341153  0.33317991  0.33340857]
07/22/2018 11:20:50 PM: epoch: 0


KeyboardInterrupt: 